In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import random
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

from tqdm import tqdm

import skimage.io
import skimage.segmentation
import skimage.morphology

import sys
__file__ = 'full_experiment.ipynb'
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

import utils.dirtools  # utils package should has __init__.py in it
import utils.augmentation
import utils.model_builder
import utils.data_provider
import utils.metrics
import utils.objectives
import utils.evaluation

import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers
import tensorflow as tf

from config import config_vars

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def empty_dir(folder):
    print('empty directory: ', folder)
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

os.environ["CUDA_VISIBLE_DEVICES"] = "3"  ### 

# build session running on GPU 1
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
# configuration.gpu_options.visible_device_list = "0, 1"
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

Using TensorFlow backend.


In [2]:
config_vars["root_directory"] = 'DATA/FISH/'
experiment_name = '26'

config_vars = utils.dirtools.setup_working_directories(config_vars)
config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)
os.makedirs(config_vars["normalized_images_dir"], exist_ok=True)
os.makedirs(config_vars["boundary_labels_dir"], exist_ok=True)

config_vars["no_boundary_labels_dir"] = 'DATA/FISH/no_boundary_labels/'
os.makedirs(config_vars["no_boundary_labels_dir"], exist_ok=True)

In [3]:
config_vars

{'root_directory': 'DATA/FISH/',
 'pixel_depth': 8,
 'min_nucleus_size': 25,
 'boundary_size': 2,
 'learning_rate': 0.0001,
 'epochs': 200,
 'steps_per_epoch': 300,
 'batch_size': 16,
 'val_batch_size': 16,
 'rescale_labels': True,
 'crop_size': 256,
 'cell_min_size': 16,
 'boundary_boost_factor': 1,
 'object_dilation': 3,
 'raw_images_dir': 'DATA/FISH/raw_images/',
 'raw_annotations_dir': 'DATA/FISH/raw_annotations/',
 'normalized_images_dir': 'DATA/FISH/norm_images/',
 'boundary_labels_dir': 'DATA/FISH/boundary_labels/',
 'experiment_dir': 'DATA/FISH/experiments/26/out/',
 'probmap_out_dir': 'DATA/FISH/experiments/26/out/prob/',
 'labels_out_dir': 'DATA/FISH/experiments/26/out/segm/',
 'path_files_training': 'DATA/FISH/experiments/26/training.txt',
 'path_files_validation': 'DATA/FISH/experiments/26/validation.txt',
 'path_files_test': 'DATA/FISH/experiments/26/test.txt',
 'model_file': 'DATA/FISH/experiments/26/model.hdf5',
 'csv_log_file': 'DATA/FISH/experiments/26/log.csv',
 'no_b

### TRAIN

#### Set Up Datasets

In [4]:
"""

'path_files_training': 'DATA/LinearTracking/training.txt',
'path_files_validation': 'DATA/LinearTracking/validation.txt',
'path_files_test': 'DATA/LinearTracking/test.txt',

folder 001 - 095 total 1585 images, 16.68 for one folder
setup dirs: `normalized` and `boundary label`
img list should contain file names like 001/0000.tif

"""  

fd_list = sorted(os.listdir('DATA/LineageTracking/raw_images/'))
# makedirs for 001, 002, ...
for f in fd_list:
    os.makedirs(config_vars["normalized_images_dir"] + f, exist_ok=True)
    os.makedirs(config_vars["boundary_labels_dir"] + f, exist_ok=True)   

"""split train, valid, test (image name list)

"""
train_fd_list = fd_list[:60]
valid_fd_list = fd_list[60:]

# boundary_labels_gai是清洗过后的labels
list_train = []
for f in train_fd_list:
    tmp_list = os.listdir('DATA/LineageTracking/boundary_labels_gai/' + f)
    tmp_list = [x for x in tmp_list if x.endswith('png')]
    for e in sorted(tmp_list):
        list_train.append(f + '/' + e)
        
list_valid = []
for f in valid_fd_list:
    tmp_list = os.listdir('DATA/LineageTracking/boundary_labels/' + f)
    tmp_list = [x for x in tmp_list if x.endswith('png')]
    for e in sorted(tmp_list):
        list_valid.append(f + '/' + e)
        
list_test = []


utils.dirtools.write_path_files(config_vars["path_files_training"], list_train)
utils.dirtools.write_path_files(config_vars["path_files_validation"], list_valid)
utils.dirtools.write_path_files(config_vars["path_files_test"], list_test)

---------------------

In [4]:
# file_list = os.listdir(config_vars["normalized_images_dir"])
# image_list = [x for x in file_list if x.endswith("png")]
# print("total images: {}".format(len(image_list)))

# set up train-valid split EVERY-TIME
def create_image_lists(dir_raw_images):
    file_list = os.listdir(dir_raw_images)
    image_list = [x for x in file_list if x.endswith("png")]
    image_list = sorted(image_list)

    image_list_train_aug = []
    image_list_test = []
#     image_list_train = []
#     image_list_validation = image_list
    
    image_list_validation = image_list[:48]
    image_list_2 = image_list[48:]
    random.shuffle(image_list_2)
    image_list_train = image_list_2
    return image_list_train, image_list_test, image_list_validation, image_list_train_aug

[list_training, list_test, list_validation, list_training_aug] = create_image_lists(
    config_vars["normalized_images_dir"],
#         config_vars["training_fraction"],
#         config_vars["validation_fraction"]
)

# write list into txt file
utils.dirtools.write_path_files(config_vars["path_files_training"], list_training)
utils.dirtools.write_path_files(config_vars["path_files_validation"], list_validation)
utils.dirtools.write_path_files(config_vars["path_files_test"], list_test)

##### data generator

In [5]:
# read txt file into dict partitions with 3 list for train/valid/test
data_partitions = utils.dirtools.read_data_partitions(config_vars, load_augmented=False)

In [6]:
# setup data-generator
train_gen = utils.data_provider.random_sample_generator(
    config_vars["normalized_images_dir"],
    config_vars["boundary_labels_dir"],  ### boundary_labels_dir no_boundary_labels_dir
    data_partitions["training"],
    config_vars["batch_size"],
    config_vars["pixel_depth"],
    config_vars["crop_size"],
    config_vars["crop_size"],
    config_vars["rescale_labels"]
)

val_gen = utils.data_provider.single_data_from_images(
     config_vars["normalized_images_dir"],
     config_vars["boundary_labels_dir"],  ### boundary_labels_dir no_boundary_labels_dir
     data_partitions["validation"],
     config_vars["val_batch_size"],
     config_vars["pixel_depth"],
     config_vars["crop_size"],
     config_vars["crop_size"],
     config_vars["rescale_labels"]
)

In [7]:
train_iter = iter(train_gen)
a = next(train_iter)
len(a[0])

Training with 291 images.


16

#### Traininig Model

In [7]:
from keras import metrics

# delete boundary, output_channel=1, activation="sigmoid"
# with boundary, output_channel=3, activation=None
model = utils.model_builder.get_model(config_vars["crop_size"], config_vars["crop_size"], 
                                      output_channel=3, activation=None) 
# softmax in the end match with crossentropy
# use sigmoid for one class?
# loss = "binary_crossentropy"
loss = utils.objectives.weighted_crossentropy

### PRINT RESULT In the TRAIN ###
# def tf_print(op, tensors, message=None):
#     def print_message(x):
#         sys.stdout.write(message + " %s\n" % x)
#         return x
#     prints = [tf.py_func(print_message, [tensor], tensor.dtype) for tensor in tensors]
#     with tf.control_dependencies(prints):
#         op = tf.identity(op)
#     return op

# def my_loss(y_true, y_pred):
#     loss = -tf.reduce_sum(y_true * tf.log(y_pred))
#     loss = tf_print(y_pred, [tf.reduce_max(y_pred)], 'y_pred max = ')
#     return loss
### PRINT RESULT In the TRAIN ###

my_metrics = [
           keras.metrics.categorical_accuracy, 
           utils.metrics.channel_recall(channel=0, name="background_recall"), 
           utils.metrics.channel_precision(channel=0, name="background_precision"),
           utils.metrics.channel_recall(channel=1, name="interior_recall"), 
           utils.metrics.channel_precision(channel=1, name="interior_precision"),
           utils.metrics.channel_recall(channel=2, name="boundary_recall"), 
           utils.metrics.channel_precision(channel=2, name="boundary_precision"),
          ]

optimizer = keras.optimizers.RMSprop(lr=config_vars["learning_rate"])

###
# model.compile(loss=loss, metrics=[metrics.binary_accuracy], optimizer=optimizer)

model.compile(loss=loss, metrics=my_metrics, optimizer=optimizer)


In [8]:
# Callbacks
log_folder = 'logs/'

csv = keras.callbacks.CSVLogger(filename=config_vars["csv_log_file"])  # append

tboard = keras.callbacks.TensorBoard(log_dir=log_folder + experiment_name, 
                                      histogram_freq=0, 
                                      batch_size=16, 
                                      write_graph=True, 
                                      write_grads=False, write_images=True,
                                      update_freq='epoch')
# add ModelCheckpoints
# monitor val-loss
weights_filename1 = log_folder + experiment_name + '/model-{epoch:02d}-{val_loss:.2f}.h5'
modelckp1 = keras.callbacks.ModelCheckpoint(weights_filename1, verbose=1, save_weights_only=True,
                                           monitor='val_loss', period=1, save_best_only=True)
weights_filename2 = log_folder + experiment_name + '/model-{epoch:02d}-{loss:.2f}.h5'
modelckp2 = keras.callbacks.ModelCheckpoint(weights_filename2, verbose=1, save_weights_only=True,
                                           monitor='loss', period=1, save_best_only=True)

# min_delta: threshold for measuring the new optimum,
#       to only focus on significant changes.
# cooldown: number of epochs to wait before resuming
#       normal operation after lr has been reduced.
reducelr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, 
                                             verbose=1, mode='min', min_lr=1e-7, 
                                             cooldown=10, min_delta=1e-4)
# min_lr could be smaller

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=15, 
                              verbose=1, mode='auto', baseline=None, restore_best_weights=False)

callbacks = [csv, tboard, modelckp1, modelckp2, reducelr, earlystop]

In [ ]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=config_vars["steps_per_epoch"],  # 500 config_vars["steps_per_epoch"]
    epochs=config_vars["epochs"],
    validation_data=val_gen,
    validation_steps= int(config_vars["steps_per_epoch"]/6),  # must bigger than val_batch_size
    callbacks=callbacks,
    verbose = 1
)
print('Done! :)')

# save one weight at the end of the training
model.save_weights(config_vars["model_file"])

Epoch 1/200
Training with 291 images.
 41/300 [===>..........................] - ETA: 3:26 - loss: 0.9037 - categorical_accuracy: 0.8586 - background_recall: 0.8581 - background_precision: 0.9891 - interior_recall: 0.8296 - interior_precision: 0.9262 - boundary_recall: 0.9585 - boundary_precision: 0.3655

### PREDICT

In [7]:
image_names = [os.path.join(config_vars["normalized_images_dir"], f) for f in data_partitions["validation"]]
imagebuffer = skimage.io.imread_collection(image_names)
images = imagebuffer.concatenate()

dim1, dim2 = images.shape[1], images.shape[2]
images = images.reshape((-1, dim1, dim2, 1))
# preprocess (assuming images are encoded as 8-bits in the preprocessing step)
images = images / 255

### build model and load weights
# model = utils.model_builder.get_model(dim1, dim2, output_channel=1, activation="sigmoid") # stupid error!
model = utils.model_builder.get_model(dim1, dim2, output_channel=3, activation=None)

model.load_weights(config_vars["model_file"])
# model.load_weights('logs/15/model-01-0.19.h5')

predictions = model.predict(images, batch_size=1)

"""prepare gt annot & bd image names

rl: abrev. for raw label
bl: abrev. for boundary label
"""
rl_names = [os.path.join(config_vars["raw_annotations_dir"], f) for f in data_partitions["validation"]]
rl_buffer = skimage.io.imread_collection(rl_names) 
bl_names = [os.path.join(config_vars["boundary_labels_dir"], f) for f in data_partitions["validation"]]
bl_buffer = skimage.io.imread_collection(bl_names) 

In [8]:
empty_dir(config_vars["probmap_out_dir"])
empty_dir(config_vars["labels_out_dir"])

empty directory:  DATA/FISH/experiments/26/out/prob/
empty directory:  DATA/FISH/experiments/26/out/segm/


In [ ]:
for i in range(len(images)):
    filename = imagebuffer.files[i]
    imgname = os.path.basename(filename)
    
    original_image = skimage.io.imread(filename)
    rl_image = skimage.io.imread(rl_buffer.files[i])
    bl_image = skimage.io.imread(bl_buffer.files[i])
    
    probmap = predictions[i].squeeze()
    os.makedirs(config_vars["probmap_out_dir"], exist_ok=True)
    skimage.io.imsave(config_vars["probmap_out_dir"] + imgname, probmap.astype('uint8'))
    
#     pred = probmap.copy()
#     thre = 0.5
#     pred[probmap >= thre] = 1
#     pred[probmap < thre] = 0
#     pred = pred.astype('uint8')
#     cell = skimage.morphology.remove_small_holes(pred, min_size=config_vars["cell_min_size"])
#     cell = skimage.morphology.remove_small_objects(cell, min_size=config_vars["cell_min_size"])
#     [label, num] = skimage.morphology.label(cell, return_num=True)
    
    pred = utils.metrics.probmap_to_pred(probmap, config_vars["boundary_boost_factor"])
    label = utils.metrics.pred_to_label(pred, config_vars["cell_min_size"])
    
    os.makedirs(config_vars["labels_out_dir"], exist_ok=True)
    skimage.io.imsave(config_vars["labels_out_dir"] + imgname, label.astype('uint8'))
    
    if (i < 15):
        f, ax = plt.subplots(2,3,figsize=(18,12))
        ax[0][0].imshow(original_image)
        ax[0][0].title.set_text('original image')
        ax[0][1].imshow(bl_image)
        ax[0][1].title.set_text('ground truth boundary')
        ax[0][2].imshow(rl_image)
        ax[0][2].title.set_text('ground truth label')
        ax[1][0].imshow(pred)
        ax[1][0].title.set_text('predict boundary')
        ax[1][1].imshow(probmap)
        ax[1][1].title.set_text('predict boundary probmap')
        ax[1][2].imshow(label)
        ax[1][2].title.set_text('predict label')
        for a in ax:
            for a_ in a:
                a_.set_xticks([])
                a_.set_yticks([])
        plt.show()
        
#     if (i == 1):
#         break


In [ ]:
"""this is for Lineage Tracking data with second order directories

"""

for i in range(len(images)):
    filename = imagebuffer.files[i]
    # imgname = os.path.basename(filename)
    filename_split = filename.split('/')
    imgname = '/' + filename_split[-2] + '/' + filename_split[-1]
    
    original_image = skimage.io.imread(filename)
    rl_image = skimage.io.imread(rl_buffer.files[i])
    bl_image = skimage.io.imread(bl_buffer.files[i])
    
    probmap = predictions[i].squeeze()
    os.makedirs(os.path.join(config_vars["probmap_out_dir"], filename_split[-2]), exist_ok=True)
    skimage.io.imsave(config_vars["probmap_out_dir"] + imgname, probmap.astype('uint8'))
    
    pred = utils.metrics.probmap_to_pred(probmap, config_vars["boundary_boost_factor"])
    label = utils.metrics.pred_to_label(pred, config_vars["cell_min_size"])
    os.makedirs(os.path.join(config_vars["labels_out_dir"], filename_split[-2]), exist_ok=True)
    skimage.io.imsave(config_vars["labels_out_dir"] + imgname, label.astype('uint8'))
    
    if (i < 15):
        f, ax = plt.subplots(2,3,figsize=(12,8))
        ax[0][0].imshow(original_image)
        ax[0][0].title.set_text('original image')
        ax[0][1].imshow(bl_image)
        ax[0][1].title.set_text('ground truth boundary')
        ax[0][2].imshow(rl_image)
        ax[0][2].title.set_text('ground truth label')
        ax[1][0].imshow(pred)
        ax[1][0].title.set_text('predict boundary')
        ax[1][1].imshow(probmap)
        ax[1][1].title.set_text('predict boundary probmap')
        ax[1][2].imshow(label)
        ax[1][2].title.set_text('predict label')
        for a in ax:
            for a_ in a:
                a_.set_xticks([])
                a_.set_yticks([])
        plt.show()
        
#     if (i == 15):
#         break


In [408]:
"""
understand sum operation
"""

a = np.arange(12).reshape(3,4)
print(a[0])
np.sum(a, axis=0)

[0 1 2 3]


array([12, 15, 18, 21])

### PRINT Results



In [ ]:
images_dirs = ['DATA/DNA_FISH/norm_images/',
              'DATA/IF_images/norm_images/',
              'DATA/Lineage_Tracking_ZY/norm_images/',
              'DATA/Lineage_Tracking_KZ/norm_images/']

labels_dirs = ['DATA/DNA_FISH/experiments/04/out/segm/',
              'DATA/IF_images/experiments/04/out/segm/',
              'DATA/Lineage_Tracking_ZY/experiments/04/out/segm/',
              'DATA/Lineage_Tracking_KZ/experiments/04/out/segm/']

In [ ]:
def difference(raw_label, pred_label):
    ground_truth = raw_label.copy()
    prediction = pred_label.copy()
    IOU = utils.evaluation.intersection_over_union(ground_truth, prediction)
    diff = np.zeros(ground_truth.shape + (3,))  # become 3 channels
    A = ground_truth.copy()
    B = prediction.copy()
    A[A > 0] = 1
    B[B > 0] = 1
    D = A - B
    #diff[D > 0,:2] = 1
    #diff[D < 0,1:] = 1
    
    # Object-level errors
    C = IOU.copy()
    threshold = 0.5  # if set to 0.8, more misses will appear, but 0.5 no miss
    C[C >= threshold] = 1
    C[C < threshold] = 0
    missed = np.where(np.sum(C, axis = 1) == 0)[0]  # for original cell, none predict cell match 
    extra = np.where(np.sum(C, axis = 0) == 0)[0]  # for predict cell, none original cell match

    for m in missed:
        diff[ground_truth == m + 1, 0] = 1
    for e in extra:
        diff[prediction == e + 1, 2] = 1
        
    return diff, str(len(missed)), str(len(extra))

In [ ]:
def compare(img_name):
    original_images = []
    pred_labels = []
    for i in range(4):
        ori_img_filename = os.path.join(images_dirs[i], img_name)
        original_image = skimage.io.imread(ori_img_filename)
    
        pred_label_filename = os.path.join(labels_dirs[i], img_name)
        pred_label = skimage.io.imread(pred_label_filename)
        
        struct = skimage.morphology.square(3)
        pred_label = skimage.morphology.dilation(pred_label, struct)
        pred_label = skimage.segmentation.relabel_sequential(pred_label)[0] #[30:-30,30:-30])[0]
            
        # make graph easier to look
        inc = lambda x: x if x == 0 else x + 100
        inc = np.vectorize(inc)
        pred_label = inc(pred_label)
        
        original_images.append(original_image)
        pred_labels.append(pred_label)
        
    fig, ax = plt.subplots(2, 4, figsize=(20,10))
    fig.suptitle(img_name)
    ax[0][0].set_title("DNA_FISH")
    ax[0][0].imshow(original_images[0])
    ax[0][1].set_title("IF_images")
    ax[0][1].imshow(original_images[1])
    ax[0][2].set_title("Linear_Tracking_zy")
    ax[0][2].imshow(original_images[2])
    ax[0][3].set_title("Linear_Tracking_kz")
    ax[0][3].imshow(original_images[3])
    
#     ax[1][0].set_title("DNA_FISH")
    ax[1][0].imshow(pred_labels[0])
#     ax[1][1].set_title("IF_images")
    ax[1][1].imshow(pred_labels[1])
#     ax[1][2].set_title("Linear_Tracking_zy")
    ax[1][2].imshow(pred_labels[2])
#     ax[1][3].set_title("Linear_Tracking_kz")
    ax[1][3].imshow(pred_labels[3])


    # plt.figure(figsize=(6,6))
    # plt.imshow(original_image)  #, cmap="nipy_spectral")
    # plt.show()

In [ ]:
for i in np.random.randint(74, size = 20):
    img = "{:04}".format(i) + ".png"
    compare(img)